In [15]:
# определения

import pandas as pd
import numpy as np
import seaborn as sns
import re
import pickle
import pymorphy2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix,roc_curve,auc,precision_recall_curve,precision_score,recall_score,f1_score,accuracy_score

from sklearn.decomposition import PCA

morph = pymorphy2.MorphAnalyzer()
vectorizer = CountVectorizer()
plt.style.use('dark_background')
# Options for pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.max_columns = 50
pd.options.display.min_rows = 30

In [3]:
# здесь "на память" описываю операторы в одну - две строчки 

# PANDAS
# data.iloc[[0,3,100]] - показать строки по индексу 0,3,100
# data[data.Survived == 0 ] - отбор по значению
# data[data.Survived == 0 ][['Sex','Age']] - отбор  и показать только выбранные колонки
# data['new_col'] = data['Survived'] - в новую колонку добавить значение старой колонки
# data.rename(columns = {'new_col':'del_col'}) - переименовать колонку
# data = data.drop(['new_col'],axis = 'columns') - удалить колонку
# del data["column"] - еще один способ удалить колонку
# print(data.groupby(['Sex','Survived'])['PassengerId'].count()) - сгруппировать 
# по колонкам 'Sex','Survived' и подсчету по колонке PassengerId
# data.isna().sum() - показать сколько пустых значений в каждой колонке
# data.Survived.value_counts() - показать сколько уникальных значений в колонке  Survived
# data.Survived.max() - показать максимальное значение в колонке  Survived
# data.Survived.nlagerset(3) - показать 3 максимальных  значения в колонке  Survived
# data.Survived.idxmax() - показать индекс максимальноого значения в колонке  Survived
# data.loc[23] - показать строку 23
# data = data.dropna() - удалить все строки с пропусками
# data.Sex.replace('male', 0, inplace=True) - заменить значения в колонке Sex: mail - 0
# test_out = pd.DataFrame({'PassengerId': test_df.index,'Survived': preds})
# создать новый dataframe для выгрузки на кагл
# data = pd.read_excel("resolution_100.xlsx") - прочитать файл excel
# data.describe() - получаем описание фрэйма, макс, мин, количество и прочее  
# data.fillna(method='ffill', inplace=True) - заполнить все пустые предыдущими значениями
# data = pandas.get_dummies(data,columns=['columns']) - раскрытие строк через доп. столбцы
# data = pd.concat([data_train,data_test]) - объединение таблиц
# data = data.assign(type_age="") - добавить колонку
# data.loc[1,"type_age"] = "a" - записать значение в колонку по строке
# def valuation_formula(x, y): # заполнение ячеек через lambda
#   return x * y * 0.5
# data['price'] = data.apply(lambda row: valuation_formula(row['x'], row['y']), axis=1)
# pandas.pivot_table - сводная таблица
# for idx,row in anime[:2].iterrows(): - обход элементов dataframe
#    print(idx, row)
# data['price'] = data['price'].apply(функция) - применение функции к столбцу
# data.map({'cat': 'kitten', 'dog': 'puppy'}) - замена значений по словарю



# SEABORN
# sns.heatmap(data.corr()) - вывести график корреляций по всем колонкам

# SKLEARN
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42) 
# разбить данные на тренировочную и тестовую выборку
# 20 процентов на тестовую , случайность выборки зафиксирована
# accuracy_score(y_test,y_pred) - коэффециент проверки на проверочной выборке
# vectorizer.get_feature_names() - посмотреть словарь векторов текста
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer, 
# стоит отметить что порядок совпадает с матрице
# text_vector_array = text_vector.toarray() - покаазть матрицу


# PICKLE
# model = pickle.load(open("filename", 'rb')) - загрузка модели из файла
# pickle.dump(model, open("filename", 'wb')) - сохрванение модели в файл

# 

In [35]:
data_cv = pd.read_csv("close_prices.csv",index_col='date')
data_dj = pd.read_csv("djia_index.csv",index_col='date')

In [47]:
data_dj

,^DJI
date,
2013-09-23,15401.380
2013-09-24,15334.590
2013-09-25,15273.260
2013-09-26,15328.300
2013-09-27,15258.240
2013-09-30,15129.670
2013-10-01,15191.700
2013-10-02,15133.140
2013-10-03,14996.480


In [14]:
data_cv[:1]

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440,117.510,85.030,24.270,125.520,59.410,64.750,24.280,165.250,75.910,190.990,23.620,89.090,51.460,38.630,97.280,121.110,47.680,32.740,68.980,28.800,79.280,34.220,86.380,71.820,109.420,196.240,47.980,76.420,87.750


In [18]:
X = np.array(data_cv)
pca = PCA(n_components=10)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [34]:
print( pca.explained_variance_ratio_[0:4].sum() )

0.9277429537836401


In [37]:
xindex1 = pd.Series( X[:,0], index=data_cv.index, name='xindex1' )

In [40]:
corr = np.corrcoef( pd.concat( [xindex1, data_dj], axis=1 ), rowvar=False )
corr

array([[1.        , 0.39432267],
       [0.39432267, 1.        ]])

In [42]:
corr = np.corrcoef( pd.concat( [xindex1, data_dj], axis=1 ).T )
corr

array([[1.        , 0.39432267],
       [0.39432267, 1.        ]])

In [44]:
print( data_cv.columns[ np.argmax( pca.components_[0] ) ] )

V


In [45]:
first_comp = pd.DataFrame(pca.transform(X)[:,0])

In [48]:
coef = np.corrcoef(first_comp.T, data_dj.T)[1,0]
print("Corrcoef %0.2f" % coef)

Corrcoef 0.91
